In [ ]:
import pandas as pd
import re


df = pd.read_csv(
    r"standardcharges.csv",
    header=2,
    encoding="latin1"
)
df.columns = df.columns.str.strip()

# Base columns that must exist
base_keys = [
    'description', 'code|1', 'code|1|type', 'setting',
    'standard_charge|min', 'standard_charge|max'
]
base_columns = [col for col in base_keys if col in df.columns]
print(f"Using base columns: {base_columns}")

merge_keys = base_columns + ['Payer Name', 'Plan Name']

# Melting Function
def extract_melted(df, charge_pattern, new_colname):
    cols = [c for c in df.columns if charge_pattern in c and '|' in c]
    if not cols:
        print(f"No columns matched pattern: {charge_pattern}")
        return pd.DataFrame()

    print(f"Melting {len(cols)} columns for: {new_colname}")
    melted = df[base_columns + cols].melt(
        id_vars=base_columns,
        value_vars=cols,
        var_name='charge_key',
        value_name=new_colname
    )

    
    payer_info = melted['charge_key'].str.extract(r'standard_charge\|(.+?)\|(.+?)\|')

    melted['Payer Name'] = payer_info[0].str.replace("_", " ", regex=False)
    melted['Plan Name'] = payer_info[1].str.replace("_", " ", regex=False)

    
    melted = melted.dropna(subset=[new_colname, 'Payer Name', 'Plan Name'])
    melted = melted.drop_duplicates(subset=merge_keys)

    print(f"Extracted {melted.shape[0]} rows for {new_colname}")
    return melted

# Melt Each Charge Type
dollar_df = extract_melted(df, 'negotiated_dollar', 'Standard Charge Dollar')
percentage_df = extract_melted(df, 'negotiated_percentage', 'Standard Charge Percentage')
estimated_df = extract_melted(df, 'estimated_amount', 'Estimated Amount')
method_df = extract_melted(df, 'methodology', 'Methodology')

# Merge All on Unique Keys
merged = dollar_df.copy()
for extra_df, col in [
    (percentage_df, 'Standard Charge Percentage'),
    (estimated_df, 'Estimated Amount'),
    (method_df, 'Methodology')
]:
    if not extra_df.empty:
        print(f"Merging: {col}")
        merged = merged.merge(
            extra_df[merge_keys + [col]].drop_duplicates(subset=merge_keys),
            on=merge_keys,
            how='left'
        )


merged.rename(columns={
    'description': 'DESCRIPTION',
    'code|1': 'CODE',
    'code|1|type': 'Code Type',
    'setting': 'SETTING',
    'standard_charge|min': 'Minimum Charge',
    'standard_charge|max': 'Maximum Charge'
}, inplace=True)


expected_cols = [
    'DESCRIPTION', 'CODE', 'Code Type', 'Minimum Charge', 'Maximum Charge',
    'SETTING', 'Payer Name', 'Plan Name',
    'Standard Charge Dollar', 'Standard Charge Percentage',
    'Estimated Amount', 'Methodology'
]
final_df = merged[[col for col in expected_cols if col in merged.columns]]


output_path = "cleaned_data.csv"
final_df.to_csv(output_path, index=False)
print(f"File saved: {output_path} ({final_df.shape[0]} rows)")


C:\Users\punya\AppData\Local\Temp\ipykernel_42068\2070153947.py:5: DtypeWarning: Columns (8,9,13,18,21,24,27,30,33,36,39,42,45,48,51,54,57,60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


✅ Using base columns: ['description', 'code|1', 'code|1|type', 'setting', 'standard_charge|min', 'standard_charge|max']
🔍 Melting 8 columns for: Standard Charge Dollar
✅ Extracted 0 rows for Standard Charge Dollar
🔍 Melting 8 columns for: Standard Charge Percentage
✅ Extracted 1873572 rows for Standard Charge Percentage
🔍 Melting 8 columns for: Estimated Amount
✅ Extracted 0 rows for Estimated Amount
🔍 Melting 8 columns for: Methodology
✅ Extracted 1873566 rows for Methodology
🔗 Merging: Standard Charge Percentage
🔗 Merging: Methodology
✅ File saved: elmhurst_cleaned.csv (0 rows)
